In [1]:
import h2o
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2ORandomForestEstimator

h2o.init()

db = pd.read_csv('dataset/db_modelli.csv').drop(columns = 'Unnamed: 0')
# codifica
db['conc1_mean'] = np.where(db['conc1_mean'].values > 1, 1, 0)

target = 'conc1_mean'
predictors = db.columns.to_list()
predictors.remove(target)
predictors.remove('test_cas')

# categorical to factors
db_h = h2o.H2OFrame(db)
# occorre
for i in db.columns:
    if db[i].dtypes == 'object':
        db_h[i] = db_h[i].asfactor()

db_h['conc1_mean'] = db_h['conc1_mean'].asfactor()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from C:\Users\Simone\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Simone\AppData\Local\Temp\tmplttlt6_h
  JVM stdout: C:\Users\Simone\AppData\Local\Temp\tmplttlt6_h\h2o_Simone_started_from_python.out
  JVM stderr: C:\Users\Simone\AppData\Local\Temp\tmplttlt6_h\h2o_Simone_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.5
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_Simone_iz5yld
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.643 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
# splitting
train, test = db_h.split_frame(ratios = [0.7])

import math

hyper_params_tune = {'max_depth' : [15,17,20,23],
                'sample_rate': [x/100. for x in range(20,101)],
                'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
                'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                'min_rows': [2**x for x in range(0,int(math.log(train.nrow,2)-1)+1)],
                'nbins': [2**x for x in range(4,11)],
                'nbins_cats': [2**x for x in range(4,13)],
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["UniformAdaptive","QuantilesGlobal","RoundRobin"]}

search_criteria_tune = {'strategy': "RandomDiscrete",
                   'max_runtime_secs': 3600,  ## limit the runtime to 60 minutes
                   'max_models': 150,  ## build no more than 100 models
                   'seed' : 1234,
                   'stopping_rounds' : 5,
                   'stopping_metric' : "rmse",
                   'stopping_tolerance': 1e-4
                   }

# gradient boosting
gbm_grid_all = H2ORandomForestEstimator(
        ntrees = 1000,
        max_runtime_secs = 3600, # 1 hour
        seed = 123,
        categorical_encoding = 'eigen')

# grid search
gall = H2OGridSearch(gbm_grid_all,
                     hyper_params_tune,
                     grid_id = 'final_grid',
                     search_criteria = search_criteria_tune)

gall.train(x=predictors, 
           y=target,
           max_runtime_secs = 3600, 
           training_frame = train)
gall

drf Grid Build progress: |████████████████████████████████████████████████| 100%
Errors/Warnings building gridsearch model

Hyper-parameter: col_sample_rate_change_per_level, 0.99
Hyper-parameter: col_sample_rate_per_tree, 0.57
Hyper-parameter: histogram_type, UniformAdaptive
Hyper-parameter: max_depth, 17
Hyper-parameter: min_rows, 4.0
Hyper-parameter: min_split_improvement, 0.0
Hyper-parameter: nbins, 64
Hyper-parameter: nbins_cats, 32
Hyper-parameter: sample_rate, 0.92
failure_details: Illegal argument(s) for DRF model: final_grid_model_93.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (12,3 KB per tree x 1000 > 1,2 MB) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_93.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (12,3 KB per tree x 1000 > 1,2 MB


Hyper-parameter: col_sample_rate_change_per_level, 0.95
Hyper-parameter: col_sample_rate_per_tree, 0.61
Hyper-parameter: histogram_type, RoundRobin
Hyper-parameter: max_depth, 23
Hyper-parameter: min_rows, 2.0
Hyper-parameter: min_split_improvement, 1e-08
Hyper-parameter: nbins, 256
Hyper-parameter: nbins_cats, 128
Hyper-parameter: sample_rate, 0.24
failure_details: Illegal argument(s) for DRF model: final_grid_model_97.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (6,4 KB per tree x 1000 > 5,1 MB) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_97.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (6,4 KB per tree x 1000 > 5,1 MB) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgumentExcepti


Hyper-parameter: col_sample_rate_change_per_level, 0.92
Hyper-parameter: col_sample_rate_per_tree, 0.55
Hyper-parameter: histogram_type, UniformAdaptive
Hyper-parameter: max_depth, 15
Hyper-parameter: min_rows, 2.0
Hyper-parameter: min_split_improvement, 1e-06
Hyper-parameter: nbins, 32
Hyper-parameter: nbins_cats, 128
Hyper-parameter: sample_rate, 0.37
failure_details: Illegal argument(s) for DRF model: final_grid_model_101.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (5,6 KB per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_101.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (5,6 KB per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgumentE


Hyper-parameter: col_sample_rate_change_per_level, 1.02
Hyper-parameter: col_sample_rate_per_tree, 0.35
Hyper-parameter: histogram_type, RoundRobin
Hyper-parameter: max_depth, 20
Hyper-parameter: min_rows, 256.0
Hyper-parameter: min_split_improvement, 0.0001
Hyper-parameter: nbins, 16
Hyper-parameter: nbins_cats, 512
Hyper-parameter: sample_rate, 0.76
failure_details: Illegal argument(s) for DRF model: final_grid_model_105.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 350  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_105.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 350  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgumentE


Hyper-parameter: col_sample_rate_change_per_level, 0.99
Hyper-parameter: col_sample_rate_per_tree, 0.48
Hyper-parameter: histogram_type, UniformAdaptive
Hyper-parameter: max_depth, 23
Hyper-parameter: min_rows, 1024.0
Hyper-parameter: min_split_improvement, 0.0
Hyper-parameter: nbins, 32
Hyper-parameter: nbins_cats, 128
Hyper-parameter: sample_rate, 0.45
failure_details: Illegal argument(s) for DRF model: final_grid_model_109.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 120  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_109.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 120  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgume

...


Hyper-parameter: col_sample_rate_change_per_level, 1.02
Hyper-parameter: col_sample_rate_per_tree, 0.48
Hyper-parameter: histogram_type, UniformAdaptive
Hyper-parameter: max_depth, 17
Hyper-parameter: min_rows, 512.0
Hyper-parameter: min_split_improvement, 1e-06
Hyper-parameter: nbins, 256
Hyper-parameter: nbins_cats, 4096
Hyper-parameter: sample_rate, 0.75
failure_details: Illegal argument(s) for DRF model: final_grid_model_121.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 274  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_121.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 274  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArg


Hyper-parameter: col_sample_rate_change_per_level, 0.99
Hyper-parameter: col_sample_rate_per_tree, 0.32
Hyper-parameter: histogram_type, RoundRobin
Hyper-parameter: max_depth, 15
Hyper-parameter: min_rows, 128.0
Hyper-parameter: min_split_improvement, 0.0001
Hyper-parameter: nbins, 16
Hyper-parameter: nbins_cats, 128
Hyper-parameter: sample_rate, 0.21
failure_details: Illegal argument(s) for DRF model: final_grid_model_125.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 270  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_125.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 270  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgumentE


Hyper-parameter: col_sample_rate_change_per_level, 1.05
Hyper-parameter: col_sample_rate_per_tree, 0.7
Hyper-parameter: histogram_type, RoundRobin
Hyper-parameter: max_depth, 17
Hyper-parameter: min_rows, 1.0
Hyper-parameter: min_split_improvement, 1e-08
Hyper-parameter: nbins, 32
Hyper-parameter: nbins_cats, 128
Hyper-parameter: sample_rate, 0.87
failure_details: Illegal argument(s) for DRF model: final_grid_model_129.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (20,1 KB per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_129.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (20,1 KB per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgumentExcep


Hyper-parameter: col_sample_rate_change_per_level, 0.99
Hyper-parameter: col_sample_rate_per_tree, 0.75
Hyper-parameter: histogram_type, QuantilesGlobal
Hyper-parameter: max_depth, 15
Hyper-parameter: min_rows, 1024.0
Hyper-parameter: min_split_improvement, 0.0001
Hyper-parameter: nbins, 32
Hyper-parameter: nbins_cats, 64
Hyper-parameter: sample_rate, 0.24
failure_details: Illegal argument(s) for DRF model: final_grid_model_133.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (  95  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_133.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (  95  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgu


Hyper-parameter: col_sample_rate_change_per_level, 0.94
Hyper-parameter: col_sample_rate_per_tree, 0.74
Hyper-parameter: histogram_type, RoundRobin
Hyper-parameter: max_depth, 23
Hyper-parameter: min_rows, 256.0
Hyper-parameter: min_split_improvement, 1e-06
Hyper-parameter: nbins, 16
Hyper-parameter: nbins_cats, 64
Hyper-parameter: sample_rate, 0.44
failure_details: Illegal argument(s) for DRF model: final_grid_model_137.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 307  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

failure_stack_traces: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DRF model: final_grid_model_137.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory ( 307  B per tree x 1000 > Zero  ) - try decreasing ntrees and/or max_depth or increasing min_rows!

	at water.exceptions.H2OModelBuilderIllegalArgumentExc

Miglior risultato da testare: 
- col_sample_rate_change_per_level: 1.08
- col_sample_rate_per_tree: 0.69
- histogram_type: RoundRobin
- max_depth: 20
- min_rows: 2
- min_split_improvement: 1e-8
- nbins: 512
- nbins_cats: 1024
- sample_rate: 0.85

In [6]:
train, test = db_h.split_frame(ratios = [0.7])

# Best model -- eigen 
drf = H2ORandomForestEstimator(nfolds = 3, ntrees = 1000, seed = 123,
                               categorical_encoding = 'eigen',col_sample_rate_change_per_level = 1.08,
                               col_sample_rate_per_tree = 0.69, histogram_type = 'RoundRobin',
                               max_depth = 20, min_rows = 2, min_split_improvement = 1e-8,
                               nbins= 512, nbins_cats = 1024, sample_rate = 0.85)

drf.train(x = predictors, y = target, training_frame = train)
perf = drf.model_performance(test_data = test)
perf

drf Model Build progress: |███████████████████████████████████████████████| 100%

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.09804376276991
RMSE: 0.31311940656866033
LogLoss: 0.32379383910294834
Mean Per-Class Error: 0.13596064990185996
AUC: 0.9314917473470816
AUCPR: 0.9551572804037541
Gini: 0.8629834946941632

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5401095430165727: 


,,0,1,Error,Rate
0,0,2097.0,483.0,0.1872,(483.0/2580.0)
1,1,434.0,4017.0,0.0975,(434.0/4451.0)
2,Total,2531.0,4500.0,0.1304,(917.0/7031.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.540110,0.897553,220.0
1,max f2,0.178505,0.931997,323.0
2,max f0point5,0.650579,0.905782,188.0
3,max accuracy,0.540110,0.869578,220.0
4,max precision,0.994402,0.995050,4.0
5,max recall,0.004083,1.000000,398.0
6,max specificity,0.999863,0.999612,0.0
7,max absolute_mcc,0.569532,0.719841,212.0
8,max min_per_class_accuracy,0.634351,0.861829,192.0
9,max mean_per_class_accuracy,0.606821,0.864039,200.0



Gains/Lift Table: Avg response rate: 63,31 %, avg score: 63,06 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010383,0.999447,1.558006,1.558006,0.986301,0.999885,0.986301,0.999885,0.016176,0.016176,55.800605,55.800605
1,,2,0.020054,0.997408,1.579645,1.568442,1.000000,0.998468,0.992908,0.999202,0.015277,0.031454,57.964502,56.844187
2,,3,0.030294,0.993030,1.579645,1.572229,1.000000,0.995356,0.995305,0.997902,0.016176,0.047630,57.964502,57.222885
3,,4,0.041246,0.989746,1.559130,1.568751,0.987013,0.990945,0.993103,0.996055,0.017075,0.064705,55.913015,56.875092
4,,5,0.051202,0.985774,1.579645,1.570869,1.000000,0.987686,0.994444,0.994427,0.015727,0.080431,57.964502,57.086922
5,,6,0.100128,0.971898,1.556685,1.563938,0.985465,0.978770,0.990057,0.986776,0.076163,0.156594,55.668507,56.393833
6,,7,0.150050,0.964128,1.516639,1.548202,0.960114,0.968199,0.980095,0.980596,0.075713,0.232307,51.663924,54.820185
7,,8,0.200256,0.956848,1.561745,1.551597,0.988669,0.960467,0.982244,0.975549,0.078409,0.310717,56.174536,55.159735
8,,9,0.300100,0.938969,1.534641,1.545956,0.971510,0.947891,0.978673,0.966347,0.153224,0.463941,53.464089,54.595591
9,,10,0.400085,0.885352,1.498753,1.534159,0.948791,0.913998,0.971205,0.953265,0.149854,0.613795,49.875282,53.415933


In [7]:
# Best Model -- One-hot-encoding
drf1 = H2ORandomForestEstimator(nfolds = 3, ntrees = 1000, seed = 123,
                               categorical_encoding = 'one_hot_explicit',col_sample_rate_change_per_level = 1.08,
                               col_sample_rate_per_tree = 0.69, histogram_type = 'RoundRobin',
                               max_depth = 20, min_rows = 2, min_split_improvement = 1e-8,
                               nbins= 512, nbins_cats = 1024, sample_rate = 0.85)

drf1.train(x = predictors, y = target, training_frame = train)
perf1 = drf1.model_performance(test_data = test)
perf1

drf Model Build progress: |███████████████████████████████████████████████| 100%

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.08445405968296356
RMSE: 0.2906098065842988
LogLoss: 0.2977533693290306
Mean Per-Class Error: 0.105148438030649
AUC: 0.9551795694374053
AUCPR: 0.9693989725720258
Gini: 0.9103591388748107

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5349970206269486: 


,,0,1,Error,Rate
0,0,2165.0,415.0,0.1609,(415.0/2580.0)
1,1,266.0,4185.0,0.0598,(266.0/4451.0)
2,Total,2431.0,4600.0,0.0969,(681.0/7031.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.534997,0.924760,208.0
1,max f2,0.414898,0.948937,249.0
2,max f0point5,0.711870,0.928193,149.0
3,max accuracy,0.544745,0.903143,205.0
4,max precision,0.983250,1.000000,0.0
5,max recall,0.043043,1.000000,392.0
6,max specificity,0.983250,1.000000,0.0
7,max absolute_mcc,0.544745,0.790034,205.0
8,max min_per_class_accuracy,0.634889,0.893023,176.0
9,max mean_per_class_accuracy,0.619395,0.894852,181.0



Gains/Lift Table: Avg response rate: 63,31 %, avg score: 62,98 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010098,0.973306,1.579645,1.579645,1.000000,0.978184,1.000000,0.978184,0.015951,0.015951,57.964502,57.964502
1,,2,0.020054,0.966792,1.579645,1.579645,1.000000,0.969742,1.000000,0.973993,0.015727,0.031678,57.964502,57.964502
2,,3,0.030010,0.961609,1.534512,1.564672,0.971429,0.964305,0.990521,0.970779,0.015277,0.046956,53.451231,56.467208
3,,4,0.040108,0.957161,1.579645,1.568442,1.000000,0.959457,0.992908,0.967928,0.015951,0.062907,57.964502,56.844187
4,,5,0.050064,0.953615,1.557079,1.566182,0.985714,0.955318,0.991477,0.965421,0.015502,0.078409,55.707867,56.618214
5,,6,0.100128,0.939067,1.570670,1.568426,0.994318,0.945654,0.992898,0.955537,0.078634,0.157043,57.066977,56.842595
6,,7,0.150050,0.927747,1.557143,1.564672,0.985755,0.933614,0.990521,0.948243,0.077735,0.234779,55.714296,56.467208
7,,8,0.200114,0.916010,1.548232,1.560559,0.980114,0.922115,0.987918,0.941706,0.077511,0.312289,54.823163,56.055905
8,,9,0.300100,0.882941,1.545940,1.555688,0.978663,0.900018,0.984834,0.927817,0.154572,0.466861,54.593994,55.568832
9,,10,0.400085,0.835472,1.487518,1.538652,0.941679,0.859965,0.974049,0.910860,0.148731,0.615592,48.751779,53.865175


Dimezziamo il numero di alberi per vedere quanto si degrada il modello.

In [4]:
train, test = db_h.split_frame(ratios = [0.7])
print(train.shape, test.shape)

# One-hot-encoding
drf1_d = H2ORandomForestEstimator(nfolds = 3, ntrees = 500, seed = 123,
                               categorical_encoding = 'one_hot_explicit',col_sample_rate_change_per_level = 1.08,
                               col_sample_rate_per_tree = 0.69, histogram_type = 'RoundRobin',
                               max_depth = 20, min_rows = 2, min_split_improvement = 1e-8,
                               nbins= 512, nbins_cats = 1024, sample_rate = 0.85)

drf1_d.train(x = predictors, y = target, training_frame = train)
perf1_d = drf1_d.model_performance(test_data = test)
perf1_d

(16394, 20) (6938, 20)
drf Model Build progress: |███████████████████████████████████████████████| 100%

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.08335884355950705
RMSE: 0.2887193162216672
LogLoss: 0.29609681983575126
Mean Per-Class Error: 0.09810686798082324
AUC: 0.95947676669859
AUCPR: 0.9716136289419546
Gini: 0.9189535333971799

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5299175315653398: 


,,0,1,Error,Rate
0,0,2157.0,409.0,0.1594,(409.0/2566.0)
1,1,221.0,4151.0,0.0505,(221.0/4372.0)
2,Total,2378.0,4560.0,0.0908,(630.0/6938.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.529918,0.929467,216.0
1,max f2,0.411085,0.950845,255.0
2,max f0point5,0.651524,0.932490,175.0
3,max accuracy,0.542314,0.909340,212.0
4,max precision,0.983440,1.000000,0.0
5,max recall,0.042004,1.000000,394.0
6,max specificity,0.983440,1.000000,0.0
7,max absolute_mcc,0.567133,0.804408,204.0
8,max min_per_class_accuracy,0.624409,0.900732,185.0
9,max mean_per_class_accuracy,0.608713,0.901893,190.0



Gains/Lift Table: Avg response rate: 63,02 %, avg score: 63,03 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010089,0.971212,1.564247,1.564247,0.985714,0.976001,0.985714,0.976001,0.015782,0.015782,56.424650,56.424650
1,,2,0.020035,0.966329,1.586917,1.575500,1.000000,0.968551,0.992806,0.972302,0.015782,0.031565,58.691674,57.550008
2,,3,0.030124,0.962209,1.586917,1.579324,1.000000,0.964152,0.995215,0.969573,0.016011,0.047575,58.691674,57.932384
3,,4,0.040069,0.957408,1.586917,1.581208,1.000000,0.959870,0.996403,0.967165,0.015782,0.063358,58.691674,58.120841
4,,5,0.050014,0.953100,1.586917,1.582343,1.000000,0.955454,0.997118,0.964836,0.015782,0.079140,58.691674,58.234350
5,,6,0.100029,0.937669,1.573197,1.577770,0.991354,0.944862,0.994236,0.954849,0.078683,0.157823,57.319700,57.777025
6,,7,0.150043,0.926262,1.568624,1.574721,0.988473,0.931733,0.992315,0.947143,0.078454,0.236276,56.862375,57.472142
7,,8,0.200058,0.912968,1.554904,1.569767,0.979827,0.919775,0.989193,0.940301,0.077768,0.314044,55.490401,56.976707
8,,9,0.300086,0.877735,1.550331,1.563288,0.976945,0.895700,0.985110,0.925434,0.155078,0.469122,55.033077,56.328830
9,,10,0.399971,0.831279,1.511349,1.550318,0.952381,0.856479,0.976937,0.908214,0.150961,0.620082,51.134928,55.031758


In [5]:
drf1_d

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1594635900789_140


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,500.0,500.0,6033597.0,20.0,20.0,20.0,225.0,1501.0,949.254




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.08780983333833028
RMSE: 0.2963272402907473
LogLoss: 0.30481586403275956
Mean Per-Class Error: 0.1117679430440639
AUC: 0.9536593146687191
AUCPR: 0.9686916776349109
Gini: 0.9073186293374382

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.544075051754679: 


,,0,1,Error,Rate
0,0,4949.0,1051.0,0.1752,(1051.0/6000.0)
1,1,654.0,9740.0,0.0629,(654.0/10394.0)
2,Total,5603.0,10791.0,0.104,(1705.0/16394.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-07-13 15:32:55,1:19:24.212,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-07-13 16:03:30,1:49:59.194,1.0,0.361894,1.872717,0.867821,0.902024,1.457835,0.163257
2,,2020-07-13 16:03:31,1:49:59.601,2.0,0.353153,1.473169,0.879323,0.909210,1.474478,0.164574
3,,2020-07-13 16:03:31,1:50:00.101,3.0,0.341317,1.574696,0.887152,0.910806,1.465215,0.149762
4,,2020-07-13 16:03:32,1:50:00.476,4.0,0.350926,1.623617,0.873884,0.893459,1.427840,0.156572
5,,2020-07-13 16:03:32,1:50:00.804,5.0,0.356417,1.393996,0.871608,0.893888,1.433869,0.166815
6,,2020-07-13 16:03:32,1:50:01.194,6.0,0.357737,1.307167,0.872305,0.895447,1.437612,0.169742
7,,2020-07-13 16:03:33,1:50:01.507,7.0,0.354756,1.172543,0.875567,0.899079,1.442331,0.167238
8,,2020-07-13 16:03:33,1:50:01.882,8.0,0.352972,1.090602,0.879003,0.901526,1.447780,0.166232
9,,2020-07-13 16:03:34,1:50:02.382,9.0,0.351631,1.088163,0.881601,0.903732,1.453273,0.164789



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,LogP,210395.687500,1.000000,0.204105
1,Mol,175273.562500,0.833066,0.170033
2,bonds_number,95979.640625,0.456186,0.093110
3,MorganDensity,86005.070312,0.408778,0.083434
4,atom_number,75435.031250,0.358539,0.073180
5,ring_number,49805.437500,0.236723,0.048316
6,alone_atom_number,40845.343750,0.194136,0.039624
7,doubleBond,34754.769531,0.165188,0.033716
8,oh_count,16581.267578,0.078810,0.016086
9,obs_duration_mean,12883.176758,0.061233,0.012498



See the whole table with table.as_data_frame()


Salvo quest'ultimo modello

In [ ]:
model_path = h2o.save_model(model=drf1_d, path="C:\\Users\\Simone\\Desktop\\MLTOX-Simone\\Modelli H2O", force=True)

Con il comando **saved_model = h2o.load_model(model_path)** posso richiamare il modello fatto. Per vedere i parametri fare **saved_model.actual_params**

In [9]:
model_path

'C:\\Users\\Simone\\Desktop\\MLTOX-Simone\\Modelli H2O\\DRF_model_python_1594635900789_140'